<h1>Putting on my chemist robe and glasses :)</h1><center><img src="https://static.displate.com/280x392/displate/2022-12-28/494bd2c86cb966c2129375457a82982f_80fd640e09154845b9ca853fda67a96d.jpg" ></center>Let us first figure out what they want from us and what we want from our code. <br><br>


The dataset comprises binary classification data representing whether small molecules bind to three different protein targets. Each entry includes SMILES representations of molecule structures and binary labels for binding to each protein target. The competition data, provided by Leash Biosciences, consists of approximately **98M** training examples per protein, **200K** validation examples per protein, and **360K** test molecules per protein. *Keep in mind that dataset is very imbalanced:  roughly 0.5% of examples are classified as binders*<br><br>

<center><h2>So, what is SMILES?</h2><img src="https://static.wixstatic.com/media/7cced3_08c22a1ad56a4d2b81c46c8d71ebc34b~mv2.gif" ></center><br>

SMILES (Simplified Molecular Input Line Entry System) is a notation system for representing chemical structures in a computer-readable format. Developed with funding from the U.S. Environmental Protection Agency, it offers a flexible and easily learned approach to representing molecules. SMILES notation follows five basic syntax rules:

* Atoms and Bonds: Atoms are represented by their atomic symbols, with lowercase letters indicating aromatic atoms. Bonds are denoted by symbols (- for single, = for double, # for triple, * for aromatic, and . for disconnected structures).
* Simple Chains: Chains of atoms are represented by combining atomic symbols and bond symbols. Hydrogen atoms are suppressed unless explicitly stated.
* Branches: Branches from chains are enclosed in parentheses and placed directly after the atom to which they are connected.
* Rings: Ring structures are identified by using numbers to denote the opening and closing ring atoms. Different numbers are used for each ring, and bond symbols may precede the ring closure number.
* Charged Atoms: Charges on atoms are indicated by placing the atom symbol within brackets, enclosing the charge.<br>


<center><h2>What are the targets?</h2></center><br>

The dataset includes three protein targets:
<h3>EPHX2 (sEH):</h3><center><img src="https://www.researchgate.net/publication/26836510/figure/fig1/AS:394310370512898@1471022326590/Pathways-of-EETs-synthesis-metabolism-and-action.png" ></center><br>
* This target refers to epoxide hydrolase 2, encoded by the EPHX2 genetic locus. Its protein product, commonly known as soluble epoxide hydrolase (sEH), is an enzyme that catalyzes certain chemical reactions and hydrolyzes phosphate groups. It is a potential drug target for conditions like high blood pressure and diabetes. The dataset includes screening data obtained from Leash Biosciences, along with structural information for model evaluation.
<h3>BRD4:</h3><center><img src="https://ars.els-cdn.com/content/image/1-s2.0-S1043661823001238-ga1.jpg" ></center><br>
* Bromodomain 4 is encoded by the BRD4 locus. Its protein product, also named BRD4, plays a role in gene transcription regulation by binding to histones in the nucleus. This protein is implicated in cancer progression, and inhibiting its activity has been explored as a therapeutic strategy. The dataset includes screening data from Leash Biosciences and structural information for model evaluation.
<h3>ALB (HSA):</h3><center><img src="https://ars.els-cdn.com/content/image/1-s2.0-S0162013419301734-gr4.jpg" ></center><br>
* Serum albumin, encoded by the ALB locus, is the most abundant protein in blood. It regulates osmotic pressure and transports various molecules, including drugs, hormones, and fatty acids. Predicting the binding of small molecules to albumin is crucial for drug development, as it impacts drug distribution and effectiveness. The dataset includes screening data from Leash Biosciences and structural information for model evaluation.


<h3>Now let's get the things done! :D</h3> <br>

We shall start with the installation of necessary tools. <br><br>
**DuckDB is an open-source, in-memory database system optimized for fast analytical querying and low memory usage. It's lightweight, embeddable, and seamlessly integrates with popular programming languages like Python and R. DuckDB excels in executing rapid SQL queries on large datasets, making it ideal for data exploration and interactive analysis tasks.**

In [1]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 59.5 MB/s eta 0:00:00


**RDKit is a widely-used open-source toolkit for cheminformatics. It offers various functions for working with chemical structures and data, including molecular representation, substructure searching, and compound library handling. RDKit is popular in pharmaceutical research and drug discovery due to its comprehensive features and easy integration with Python.**

In [2]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 40.2 MB/s eta 0:00:00


In [3]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import duckdb
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import average_precision_score

In [4]:
%%time
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 30000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 30000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2min 48s, sys: 9.32 s, total: 2min 57s
Wall time: 53.8 s


In [5]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,101105739,O=C(N[C@@H](Cc1ccc(I)cc1)C(=O)O)OCC1c2ccccc2-c...,Nc1cc(N2CCNCC2)ccc1[N+](=O)[O-],Nc1nc(-c2ccc(Cl)cc2)cs1,O=C(N[Dy])[C@H](Cc1ccc(I)cc1)Nc1nc(Nc2nc(-c3cc...,BRD4,0
1,205302122,O=C(Nc1ccc([N+](=O)[O-])cc1C(=O)O)OCC1c2ccccc2...,Cl.Cl.NCCc1nc2c(s1)COCC2,COC(=O)c1nccnc1N,COC(=O)c1nccnc1Nc1nc(NCCc2nc3c(s2)COCC3)nc(Nc2...,sEH,0
2,41176303,COc1nccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Nc1cc(F)c(F)cc1[N+](=O)[O-],CCn1cc(N)c(C)n1,CCn1cc(Nc2nc(Nc3cc(F)c(F)cc3[N+](=O)[O-])nc(Nc...,HSA,0
3,288656505,O=C(O)[C@H]1CC2CCCCC2N1C(=O)OCC1c2ccccc2-c2ccc...,Nc1nc(-c2ccccc2Cl)cs1,Cc1cccc(CCCN)n1,Cc1cccc(CCCNc2nc(Nc3nc(-c4ccccc4Cl)cs3)nc(N3C4...,BRD4,0
4,294705555,[N-]=[N+]=NCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc...,Cl.NCC(F)(F)C(F)(F)F,Cl.NCC1CC(CC(N)=O)CO1,[N-]=[N+]=NCCC[C@H](Nc1nc(NCC2CC(CC(N)=O)CO2)n...,BRD4,0


Further, we convert SMILES representations of molecules into RDKit molecule objects and then generates ECFP fingerprints for each molecule, storing the results in the DataFrame. 

**ECFP, or Extended Connectivity Fingerprints**, are molecular fingerprints used in cheminformatics to encode structural features of molecules. They represent connectivity patterns of atoms by hashing local environments within a specified radius. ECFP fingerprints are widely used for similarity searching and activity prediction in drug discovery and cheminformatics due to their efficiency and robustness.

<center><img src="https://miro.medium.com/max/652/1*YjmaYuWldV2yFH1TvPvNNw.jpeg" ></center><br>

In [6]:
%%time
# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)

CPU times: user 1min 32s, sys: 2.12 s, total: 1min 34s
Wall time: 1min 34s


In [7]:
df['ecfp'].head()

0    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...
Name: ecfp, dtype: object

In [8]:
RANDOM_STATE = 42

In [9]:
%%time
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

CPU times: user 1.36 s, sys: 388 ms, total: 1.75 s
Wall time: 1.75 s


In [10]:
NUMBER_OF_MODELS = 3
# Create and train the ensemble models
catboost_model = CatBoostClassifier(iterations=500, random_state=RANDOM_STATE)
lgbm_model = LGBMClassifier(random_state=RANDOM_STATE)
xgb_model = XGBClassifier(random_state=RANDOM_STATE)

catboost_model.fit(X_train, y_train)
lgbm_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_proba_catboost = catboost_model.predict_proba(X_test)[:, 1]  # Probability of the positive class
y_pred_proba_lgbm = lgbm_model.predict_proba(X_test)[:, 1]  # Probability of the positive class
y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Average the predictions from the ensemble
y_pred_proba_ensemble = (y_pred_proba_catboost + y_pred_proba_lgbm + y_pred_proba_xgb) / NUMBER_OF_MODELS

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba_ensemble)
print(f"Mean Average Precision (mAP): {map_score:.8f}")

Learning rate set to 0.101594
0:	learn: 0.6567516	total: 116ms	remaining: 57.7s
1:	learn: 0.6278844	total: 172ms	remaining: 42.8s
2:	learn: 0.6040892	total: 225ms	remaining: 37.3s
3:	learn: 0.5867042	total: 270ms	remaining: 33.5s
4:	learn: 0.5714559	total: 321ms	remaining: 31.8s
5:	learn: 0.5589494	total: 367ms	remaining: 30.2s
6:	learn: 0.5475792	total: 414ms	remaining: 29.1s
7:	learn: 0.5357710	total: 466ms	remaining: 28.7s
8:	learn: 0.5271660	total: 514ms	remaining: 28s
9:	learn: 0.5186294	total: 562ms	remaining: 27.5s
10:	learn: 0.5118024	total: 610ms	remaining: 27.1s
11:	learn: 0.5072489	total: 656ms	remaining: 26.7s
12:	learn: 0.5005235	total: 706ms	remaining: 26.4s
13:	learn: 0.4951770	total: 754ms	remaining: 26.2s
14:	learn: 0.4899843	total: 805ms	remaining: 26s
15:	learn: 0.4850205	total: 851ms	remaining: 25.7s
16:	learn: 0.4798242	total: 899ms	remaining: 25.5s
17:	learn: 0.4761398	total: 945ms	remaining: 25.3s
18:	learn: 0.4728933	total: 992ms	remaining: 25.1s
19:	learn: 0.46

In [11]:
import os
# Process the test.parquet file chunk by chunk sinc the file is huge
test_file = '/kaggle/input/leash-BELKA/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file



# Read the test.csv file into a pandas DataFrame
# Wrap the loop with tqdm to display progress
for df_test in tqdm(pd.read_csv(test_file, chunksize=10_000)):
    
    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]

    # Predict the probabilities using ensemble models
    probabilities_catboost = catboost_model.predict_proba(X_test)[:, 1]
    probabilities_lgbm = lgbm_model.predict_proba(X_test)[:, 1]
    probabilities_xgb = xgb_model.predict_proba(X_test)[:, 1]

    # Average the predictions from the ensemble
    probabilities_ensemble = (probabilities_catboost + probabilities_lgbm + probabilities_xgb) / NUMBER_OF_MODELS
    
     # Append the probabilities to the list

# Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities_ensemble})

# Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))

168it [1:15:25, 26.94s/it]
